# SysML v2 State Machine Simulation

This example shows how to use Python SXCML library (https://github.com/Open-MBEE/scxml4py)
to simulate a state machine defined in a SysML file.

If VSCode asks you to choose the Python interpreter and you are using a virtual
environment, choose the one which includes `.venv` in its path.


Import all dependencies needed for the example.


In [ ]:
from sysmlv2x_lib.sysmlv2x import SysMLv2ToSCXML
import syside_license
syside_license.check('key')  # Validates your license"

import syside
from sysmlv2x_lib.syside_helpers import (
    get_node,
    set_feature_value,
    pprint_sysml,
)

## Convert SysMLv2 to SCXML with Syside

In [ ]:
(model, diagnostics) = syside.load_model(["simpleSM.sysml"])
assert not diagnostics.contains_errors(warnings_as_errors=True)
state_machine_node = get_node(model, ["Demo", "A_behavior"]).cast(syside.StateDefinition)
converter = SysMLv2ToSCXML(model, state_machine_node)
simpleSM_string = converter.to_xml_string()
print(simpleSM_string)

# Set up SCXML simulation environment

In [ ]:
import logging
import time

import scxml4py.helper
from scxmlApp.application import Application
from scxml4py.action import Action
from scxml4py.activity import ThreadedActivity
from scxml4py.event import Event

## Declare SCXML Listeners and do actions

In [9]:

# Installed as status listener, engine looks for "ActionStatusListener" action
class ActionStatusListener(Action):
    # implemented by the developer, stub can be generated
    def __init__(self, theData):
        Action.__init__(self, "ActionStatusListener", None, theData)
    
    def execute(self, status):
        logging.getLogger("scxml4py").info(">>>>ActionStatusListener::nb Status: <" + scxml4py.helper.formatStatus(status) + ">")

class ActionEventListener(Action):
    # implemented by the developer, stub can be generated
    def __init__(self, theData):
        Action.__init__(self, "ActionEventListener", None, theData)
    
    def execute(self, event):
        logging.getLogger("scxml4py").info(">>>>ActionEventListener::nb Event: <" + ">")
        print (event.getStatus)

# Do Action that gets invoked when entering state, name must must do action in v2 model
class ActivityOnline(ThreadedActivity):
    # implemented by the developer, stub can be generated
    def __init__(self, theEventQueue, theData):
        ThreadedActivity.__init__(self, "ActivityOnline", theEventQueue, theData)

    def run(self):
        counter = 0
        while self.isRunning() == True:
            logging.getLogger("scxml4py").info("Activity <" + self.getId() + f"> is running...{counter}")
            time.sleep(2)
            counter += 1
            if counter == 5:
                self.sendInternalEvent(Event("GoOffline"))
                self.setRunning(False)
                break
            
# Data Object, can be generated from attributes of owning part that exhibits behavior
class Data(object):
    # implemented by the developer
    # data shared between actions and activities
    def __init__(self):
        self.mSharedInfo = None
    
    def getSharedInfo(self):
        # requires mutex
        return self.mSharedInfo
    
    def setSharedInfo(self, sharedInfo):
        # requires mutex
        self.mSharedInfo = sharedInfo

## Run the Simulation

In [10]:
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(threadName)s - %(module)s - %(funcName)s - %(message)s', level=logging.DEBUG)
simple_sm_data = Data()
simple_sm = Application(simpleSM_string, actions = [ActionStatusListener,ActionEventListener], activities = [ActivityOnline], data = simple_sm_data)
simple_sm.start()
simple_sm.send_signal("GoOnline", True)
status = simple_sm.get_current_status()
print (">>>>>>" + status)
time.sleep(10)
simple_sm.send_signal("GoOffline", True)
status = simple_sm.get_current_status()
print (">>>>>>" + status)
simple_sm.terminate()

2025-08-14 16:02:27,495 - INFO - MainThread - application - __init__ - _Application::Loading SCXML model
2025-08-14 16:02:27,496 - DEBUG - MainThread - reader - parseScxml - Version: 1.0
2025-08-14 16:02:27,496 - DEBUG - MainThread - reader - parseScxml - Optional name attribute not available.
2025-08-14 16:02:27,497 - DEBUG - MainThread - reader - parseStates - Found new state: OFF
2025-08-14 16:02:27,497 - DEBUG - MainThread - reader - parseStates - Found new state: ONLINE
2025-08-14 16:02:27,497 - DEBUG - MainThread - reader - parseScxmlInitialTransition - Found initial transition to state: OFF
2025-08-14 16:02:27,498 - DEBUG - MainThread - reader - parseTransitions - Found new transition to state: ONLINE
2025-08-14 16:02:27,498 - DEBUG - MainThread - reader - parseTransitions - Found new transition to state: OFF
2025-08-14 16:02:27,500 - DEBUG - MainThread - reader - parseInvokes - Found activity ActivityOnline for state ONLINE
2025-08-14 16:02:27,500 - DEBUG - MainThread - applica

>>>>>>ONLINE
<bound method Event.getStatus of <scxml4py.event.Event object at 0x00000223AF763B10>>


2025-08-14 16:02:29,516 - INFO - Thread-11 (run) - 4202266828 - run - Activity <ActivityOnline> is running...1
2025-08-14 16:02:31,519 - INFO - Thread-11 (run) - 4202266828 - run - Activity <ActivityOnline> is running...2
2025-08-14 16:02:33,521 - INFO - Thread-11 (run) - 4202266828 - run - Activity <ActivityOnline> is running...3
2025-08-14 16:02:35,522 - INFO - Thread-11 (run) - 4202266828 - run - Activity <ActivityOnline> is running...4
2025-08-14 16:02:37,512 - DEBUG - Thread-10 - application - run - _Application::Application received event = <GoOffline>
2025-08-14 16:02:37,513 - DEBUG - Thread-10 - executor - processEvent - Adding event <GoOffline> to the external queue
2025-08-14 16:02:37,513 - DEBUG - Thread-10 - executor - processEvents - Processing external event <GoOffline>
2025-08-14 16:02:37,514 - DEBUG - Thread-10 - executor - selectTransitions - Selected transitions on event <GoOffline>:
FromState <ONLINE> ToState <OFF>  Guards <> Event <GoOffline> Actions <>

2025-08-14 

>>>>>>ONLINE
<bound method Event.getStatus of <scxml4py.event.Event object at 0x00000223AF5F7530>>
<bound method Event.getStatus of <scxml4py.event.Event object at 0x00000223AF75E950>>
